<a href="https://colab.research.google.com/github/AmedeeRoy/BirdDL/blob/main/dt_fcnet_10s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dive Prediction - Deep Network

*Predicting Seabird Diving Behaviour from GPS data*

This notebook trains a neural network to predict seabirds' dives.

Networks' characteristics:

* *Trajectory window* : 100 x 5s
* *Output resolution*: 5s
* *Representation of trajectories* : Time series
* *Layers* : fully connected

## connect to drive

In [1]:
# mount google drive
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [2]:
%cd drive/My\ Drive/

/content/drive/My Drive


## load data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils.trip import * 

In [16]:
data_train = pd.read_csv('./data/data_train.csv')
data_validation = pd.read_csv('./data/data_validation.csv')
data_test = pd.read_csv('./data/data_test.csv')

## choose resolution

In [25]:
resolution = 5

data_train_new = standardize_data(data_train)
data_validation_new = standardize_data(data_validation)
data_test_new = standardize_data(data_test)

data_train_new.head()

,trip,datetime,lon,lat,pressure,gaps,step_speed,step_direction,dive,lon_std,lat_std,step_speed_std,step_direction_cos,step_direction_sin
2,P1108_46_SV_T1,2008-12-05 15:12:41,-77.262442,-11.773072,-0.26,False,7.224129,3.984122,0,1.452647,1.542331,0.246157,0.997583,0.069480
3,P1108_46_SV_T1,2008-12-05 15:12:42,-77.262447,-11.773147,-0.26,False,8.366185,13.285157,0,1.452565,1.541362,0.285681,0.973238,0.229798
4,P1108_46_SV_T1,2008-12-05 15:12:43,-77.262482,-11.773217,-0.22,False,8.675223,22.346828,0,1.451995,1.540458,0.296376,0.924899,0.380212
5,P1108_46_SV_T1,2008-12-05 15:12:44,-77.262517,-11.773293,-0.29,False,9.279737,-1.813227,0,1.451424,1.539476,0.317297,0.999499,-0.031641
6,P1108_46_SV_T1,2008-12-05 15:12:45,-77.262518,-11.773372,-0.19,False,8.794348,-23.557701,0,1.451408,1.538456,0.300499,0.916658,-0.399672


In [26]:
data_train_new = change_resolution(data_train_new, resolution)
data_validation_new = change_resolution(data_validation_new, resolution)
data_test_new = change_resolution(data_test_new, resolution)

data_train_new.head()

,trip,datetime,lon,lat,dive
0,P1108_46_SV_T1,2008-12-05 15:12:41,-77.262442,-11.773072,0
1,P1108_46_SV_T1,2008-12-05 15:12:46,-77.262518,-11.773463,0
2,P1108_46_SV_T1,2008-12-05 15:12:51,-77.262460,-11.773983,0
3,P1108_46_SV_T1,2008-12-05 15:12:56,-77.262732,-11.774355,0
4,P1108_46_SV_T1,2008-12-05 15:13:01,-77.262960,-11.774725,0


## choose window

In [20]:
# Hyperparameters
window = 100
variable = ('lon', 'lat', 'lon_std', 'lat_std', 'step_speed_std', 'step_direction_cos', 'step_direction_sin')

train_set = TrajDataSet(data_train_new, window, variable)

In [23]:
data_train_new.head()

,trip,datetime,lon,lat,dive
0,P1108_46_SV_T1,2008-12-05 15:12:41,-77.262442,-11.773072,0
1,P1108_46_SV_T1,2008-12-05 15:12:46,-77.262518,-11.773463,0
2,P1108_46_SV_T1,2008-12-05 15:12:51,-77.262460,-11.773983,0
3,P1108_46_SV_T1,2008-12-05 15:12:56,-77.262732,-11.774355,0
4,P1108_46_SV_T1,2008-12-05 15:13:01,-77.262960,-11.774725,0


In [21]:
plt.figure(figsize=(16, 10))
idx = np.random.randint(0, len(train_set), 4)

k = -1
for i in idx:
    k += 1
    traj, matrix_dist, dive = train_set[i]

    plt.subplot(2,4, k+1)
    plt.imshow(matrix_dist)

    plt.subplot(2,4, k+5)
    plt.plot(traj[0,:], traj[1,:])
    plt.scatter(traj[0, np.array(dive, dtype = 'bool')], traj[1,np.array(dive, dtype = 'bool')], c='orange')

KeyError: ignored

<Figure size 1152x720 with 0 Axes>

# Neural Network

In [11]:
# hyperparameters
batch_size = 64

## reduce size dataset
train_set = TrajDataSet(data_train_new, window, variable, transform = ToTensor())
validation_set = TrajDataSet(data_validation_new, window, variable, transform = ToTensor())

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=True)
validation_loader = DataLoader(validation_set, batch_size=batch_size, num_workers = 0, shuffle = True, drop_last=True)

In [ ]:
class FCNet(nn.Module):

    def __init__(self):
        super(FCNet, self).__init__()

        self.cnn = nn.Sequential(
            nn.BatchNorm1d(8),
            nn.Conv1d(8, 4, kernel_size = 11, stride = 1, padding = 5, dilation = 1),
            nn.ReLU(),
            nn.Conv1d(4, 2, kernel_size = 11, stride = 1, padding = 5, dilation = 1),
            nn.ReLU(),
            nn.Conv1d(2, 1, kernel_size = 11, stride = 1, padding = 5, dilation = 1)
        )

    def forward(self, x):


        return out

def get_score(out, y):
    out, y = out.cpu(), y.cpu()
    out = 1*(out>0)
    true_positive = np.mean(out[y == True].numpy()) 
    true_negative = 1-np.mean(out[y == False].numpy())
    
    return (round(true_positive*100) , round(true_negative*100))

In [ ]:
# get sample
x, y = next(iter(train_loader)) 
weight = torch.FloatTensor([30])

# Forward model
model = UNet()

out = model(x)

# Loss and score
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss(pos_weight = weight)
criterion(out, y)
get_score(out, y)

(58.0, 46.0)

In [ ]:
## get Loss weight
torch.sum(y==y)/torch.sum(y)

tensor(42.6667)

## Training

In [ ]:
# switch to GPU
model = UNet()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
weight = weight.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight = weight)

In [ ]:
# Train the model
nb_epoch = 5

global_loss_train = []
global_loss_validation = []
    
for epoch in range(nb_epoch):
    learning_rate /= 10
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    i = 0
    for batch, (x, y) in enumerate(train_loader):
        i+=1
        # send to GPU
        x, y = x.to(device), y.to(device)

        # Run the forward pass
        out = model(x)
        loss = criterion(out, y)
        
        # Backprop and perform optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    ### Evaluation + Validation every epoch
    model.eval()
    with torch.no_grad():      
        j = 0
        # evaluation
        list_loss_train = []
        list_score_train = []
        for batch, (x, y) in enumerate(train_loader):
            j+= 1
            # send to GPU
            x, y = x.to(device), y.to(device)

            # Run the forward pass
            out = model(x)
            loss = criterion(out, y)
            score = get_score(out,y)
            list_loss_train.append(loss.item())
            list_score_train.append(score)
        
        train_loss = np.mean(list_loss_train)
        train_trueP = np.mean([tp for (tp, tn) in list_score_train])
        train_trueN = np.mean([tn for (tp, tn) in list_score_train])
            
        k = 0
        # validation
        list_loss_validation = []
        list_score_validation = []
        for batch, (x, y) in enumerate(validation_loader):
            k+= 1
            # send to GPU
            x, y = x.to(device), y.to(device)

            # Run the forward pass
            out = model(x)
            loss = criterion(out, y)
            score = get_score(out,y)
            list_loss_validation.append(loss.item())
            list_score_validation.append(score)

        validation_loss = np.mean(list_loss_validation)
        validation_trueP = np.mean([tp for (tp, tn) in list_score_validation])
        validation_trueN = np.mean([tn for (tp, tn) in list_score_validation])

    print('Epoch [{}/{}] -------------------------------------------------------------------------------------'
          .format(epoch+1, nb_epoch))
    print('Train Loss: {}, Train True Positive : {} %, Train True Negative : {} %'
            .format(round(train_loss, 2), round(train_trueP, 2), round(train_trueN, 2)))
    print('Validation Loss: {}, Validation True Positive : {} %, Validation True Negative : {} %'
            .format(round(validation_loss, 2), round(validation_trueP, 2), round(validation_trueN, 2)))
    model.train()
    
    global_loss_train.append(train_loss)
    global_loss_validation.append(validation_loss)

Epoch [1/5] -------------------------------------------------------------------------------------
Train Loss: 0.56, Train True Positive : 90.06 %, Train True Negative : 80.37 %
Validation Loss: 0.81, Validation True Positive : 80.29 %, Validation True Negative : 79.26 %
Epoch [2/5] -------------------------------------------------------------------------------------
Train Loss: 0.53, Train True Positive : 93.01 %, Train True Negative : 79.76 %
Validation Loss: 0.78, Validation True Positive : 82.88 %, Validation True Negative : 78.64 %
Epoch [3/5] -------------------------------------------------------------------------------------
Train Loss: 0.54, Train True Positive : 93.29 %, Train True Negative : 78.76 %
Validation Loss: 0.82, Validation True Positive : 82.58 %, Validation True Negative : 77.48 %
Epoch [4/5] -------------------------------------------------------------------------------------
Train Loss: 0.56, Train True Positive : 91.16 %, Train True Negative : 79.69 %
Validation